# 期末

In [2]:
import pandas as pd
import numpy as np

任务5只完成部分，后续逐渐增加

# 【任务四】显卡日志

下面给出了3090显卡的性能测评日志结果，每一条日志有如下结构：

`
Benchmarking #2# #4# precision type #1#
#1#  model average #2# time :  #3# ms
`

其中#1#代表的是模型名称，#2#的值为train(ing)或inference，表示训练状态或推断状态，#3#表示耗时，#4#表示精度，其中包含了float, half, double三种类型，下面是一个具体的例子：

`Benchmarking Inference float precision type resnet50
resnet50  model average inference time :  13.426570892333984 ms
`

请把日志结果进行整理，变换成如下状态，model_i用相应模型名称填充，按照字母顺序排序，数值保留三位小数：
<img src="../source/_static/期末-任务4.png" width="80%">

In [13]:
# pands 读取数据以及裁剪
df4 = pd.read_table('../data/测试/benchmark.txt',header=None)
df4 = df4.iloc[10:-2].reset_index(drop=True)
df4

,0
0,Benchmarking Training float precision type mna...
1,mnasnet0_5 model average train time : 28.527...
2,Benchmarking Training float precision type mna...
3,mnasnet0_75 model average train time : 34.10...
4,Benchmarking Training float precision type mna...
...,...
379,shufflenet_v2_x1_0 model average inference ti...
380,Benchmarking Inference double precision type s...
381,shufflenet_v2_x1_5 model average inference ti...
382,Benchmarking Inference double precision type s...


In [68]:
# 拼接数据
df4_1=df4[df4.index%2==0].reset_index(drop=True)
df4_2=df4[df4.index%2==1].reset_index(drop=True)
df4_all = pd.concat([df4_1, df4_2], 1)
df4_all.columns = ['process','time']
df4_all.head(3)

,process,time
0,Benchmarking Training float precision type mna...,mnasnet0_5 model average train time : 28.527...
1,Benchmarking Training float precision type mna...,mnasnet0_75 model average train time : 34.10...
2,Benchmarking Training float precision type mna...,mnasnet1_0 model average train time : 34.313...


In [69]:
# 利用正则处理成模型、运行状态、运行精度、运行时间的表
pat1 = 'Benchmarking (?P<过程>\w+\s)(?P<精度>\w+\s)precision type (?P<模型>\w+)'
# pat2 = '(?P<模型2>\w+)  model average (?P<过程2>\w+\s)time :  (\d+\.\d+)'
pat2 = '(?P<t>\d+\.\d+)'
df4_all = pd.concat([df4_all, df4_all.process.str.extract(pat1),df4_all.time.str.extract(pat2)], 1)
df4_all.head()

,process,time,过程,精度,模型,t
0,Benchmarking Training float precision type mna...,mnasnet0_5 model average train time : 28.527...,Training,float,mnasnet0_5,28.527636528015137
1,Benchmarking Training float precision type mna...,mnasnet0_75 model average train time : 34.10...,Training,float,mnasnet0_75,34.10548686981201
2,Benchmarking Training float precision type mna...,mnasnet1_0 model average train time : 34.313...,Training,float,mnasnet1_0,34.31377410888672
3,Benchmarking Training float precision type mna...,mnasnet1_3 model average train time : 35.556...,Training,float,mnasnet1_3,35.556888580322266
4,Benchmarking Training float precision type res...,resnet18 model average train time : 18.66008...,Training,float,resnet18,18.660082817077637


In [72]:
# 拼接状态与精度
df4_all.t = df4_all.t.astype(float).round(2)
df4_all['detail'] = df4_all['过程'] + '_'+ df4_all['精度']
df4_all.head()

,process,time,过程,精度,模型,t,detail
0,Benchmarking Training float precision type mna...,mnasnet0_5 model average train time : 28.527...,Training,float,mnasnet0_5,28.53,Training _float
1,Benchmarking Training float precision type mna...,mnasnet0_75 model average train time : 34.10...,Training,float,mnasnet0_75,34.11,Training _float
2,Benchmarking Training float precision type mna...,mnasnet1_0 model average train time : 34.313...,Training,float,mnasnet1_0,34.31,Training _float
3,Benchmarking Training float precision type mna...,mnasnet1_3 model average train time : 35.556...,Training,float,mnasnet1_3,35.56,Training _float
4,Benchmarking Training float precision type res...,resnet18 model average train time : 18.66008...,Training,float,resnet18,18.66,Training _float
...,...,...,...,...,...,...,...
187,Benchmarking Inference double precision type m...,mobilenet_v2 model average inference time : ...,Inference,double,mobilenet_v2,25.48,Inference _double
188,Benchmarking Inference double precision type s...,shufflenet_v2_x0_5 model average inference ti...,Inference,double,shufflenet_v2_x0_5,11.50,Inference _double
189,Benchmarking Inference double precision type s...,shufflenet_v2_x1_0 model average inference ti...,Inference,double,shufflenet_v2_x1_0,12.89,Inference _double
190,Benchmarking Inference double precision type s...,shufflenet_v2_x1_5 model average inference ti...,Inference,double,shufflenet_v2_x1_5,21.52,Inference _double


In [81]:
# 转换成宽表
ans = df4_all.pivot_table(index = '模型',
               columns = 'detail',
               values = 't',
               aggfunc = 'mean')   # ag
ans.index.name = ''
ans.columns.name = ''
ans.head()

,Inference _double,Inference _float,Inference _half,Training _double,Training _float,Training _half
,,,,,,
densenet121,144.11,15.64,19.77,417.21,93.36,88.98
densenet161,511.18,31.75,27.55,1290.29,136.62,144.32
densenet169,175.81,21.60,26.37,511.40,104.84,121.56
densenet201,223.96,26.17,33.39,654.37,129.33,118.94
mnasnet0_5,11.87,8.04,6.93,48.23,28.53,27.20


# 【任务五】水压站点的特征工程

df1和df2中分别给出了18年和19年各个站点的数据，其中列中的H0至H23分别代表当天0点至23点；df3中记录了18-19年的每日该地区的天气情况，请完成如下的任务：

`
import pandas as pd
import numpy as np
df1 = pd.read_csv('yali18.csv')
df2 = pd.read_csv('yali19.csv')
df3 = pd.read_csv('qx1819.csv')
`


* 通过df1和df2构造df，把时间设为索引，第一列为站点编号，第二列为对应时刻的压力大小，排列方式如下（压力数值请用正确的值替换）：

`
                       站点    压力
2018-01-01 00:00:00       1    1.0
2018-01-01 00:00:00       2    1.0
...                     ...    ...
2018-01-01 00:00:00      30    1.0
2018-01-01 01:00:00       1    1.0
2018-01-01 01:00:00       2    1.0
...                     ...    ...
2019-12-31 23:00:00      30    1.0
`

In [84]:
df5_1 = pd.read_csv('../data/测试/水压站点/yali18.csv')
df5_1.head(3)

,Time,MeasName,H0,H1,H2,H3,H4,H5,H6,H7,...,H14,H15,H16,H17,H18,H19,H20,H21,H22,H23
0,2018-01-01,站点4,0.402750,0.407625,0.418125,0.425250,0.426000,0.425250,0.417375,0.426375,...,0.348750,0.359250,0.355500,0.344250,0.352125,0.356250,0.34725,0.343875,0.356625,0.418875
1,2018-01-01,站点7,0.214375,0.226750,0.232375,0.233125,0.235000,0.232750,0.230875,0.220000,...,0.187375,0.196750,0.199750,0.192250,0.186250,0.183250,0.17725,0.163375,0.165250,0.199375
2,2018-01-01,站点22,0.247000,0.248125,0.271375,0.251125,0.272125,0.256375,0.257125,0.242500,...,0.245500,0.242875,0.238375,0.230875,0.237250,0.236875,0.23650,0.236500,0.241000,0.254500


In [85]:
df5_2 = pd.read_csv('../data/测试/水压站点/yali19.csv')
df5_2.head(3)

,Time,MeasName,H0,H1,H2,H3,H4,H5,H6,H7,...,H14,H15,H16,H17,H18,H19,H20,H21,H22,H23
0,2019-01-01,站点4,0.342000,0.429375,0.429000,0.440250,0.445875,0.444750,0.417750,0.387000,...,0.319875,0.326250,0.323625,0.322500,0.309000,0.307125,0.307125,0.307125,0.307125,0.307125
1,2019-01-01,站点7,0.215125,0.239500,0.257500,0.246250,0.275125,0.264625,0.229375,0.205375,...,0.180625,0.176500,0.181375,0.155125,0.159625,0.146125,0.144625,0.135250,0.158875,0.184750
2,2019-01-01,站点22,0.244750,0.248875,0.246625,0.247375,0.247375,0.245500,0.244000,0.239500,...,0.238000,0.236125,0.235375,0.238000,0.231250,0.232375,0.226750,0.227875,0.236125,0.242125


In [88]:
# 对表1、2操作，宽表到长表
df5_1_m = df5_1.melt(id_vars = ['Time', 'MeasName'],
                    value_vars = ['H'+str(i) for i in range(24)],
                    var_name = '时',
                    value_name = '压力')
df5_2_m = df5_2.melt(id_vars = ['Time', 'MeasName'],
                    value_vars = ['H'+str(i) for i in range(24)],
                    var_name = '时',
                    value_name = '压力')

In [115]:
# 表1、2 拼接 + 格式修改
df5_a_m = pd.concat([df5_1_m, df5_2_m])
df5_a_m['时'] = df5_a_m['时'].str[1:].str.pad(2,'left','0')
df5_a_m['Time'] = df5_a_m['Time'] + ' ' + df5_a_m['时']
df5_a_m['Time'] = pd.to_datetime(df5_a_m['Time'],format='%Y\\%m\\%d %H')
df5_a_m = df5_a_m.set_index('Time')
df5_a_m['站点'] = df5_a_m.MeasName.str.extract('(?P<站点>\d+)')
df5_a_m = df5_a_m[['站点','压力']]
df5_a_m.index.name = ''

In [116]:
df5_a_m.head()

,站点,压力
,,
2018-01-01,4,0.402750
2018-01-01,7,0.214375
2018-01-01,22,0.247000
2018-01-01,21,0.284250
2018-01-01,20,0.292875


In [ ]:
* 在上一问构造的df基础上，构造下面的特征序列或DataFrame，并把它们逐个拼接到df的右侧

    * 当天最高温、最低温和它们的温差
    * 当天是否有沙暴、是否有雾、是否有雨、是否有雪、是否为晴天
    * 选择一种合适的方法度量雨量/下雪量的大小（构造两个序列分别表示二者大小）
    * 限制只用4列，对风向进行0-1编码（只考虑风向，不考虑大小）

In [165]:
df5_3 = pd.read_csv('../data/测试/水压站点/qx1819.csv')
df5_3.head(2)

,日期,天气,气温,风向
0,2018-01-01,多云,1C～-4C,东南风 微风
1,2018-01-02,阴转多云,8C～0C,东北风 3-4级


In [166]:
# 正则获取最高温、最低温
pat1 = '(?P<最高温>\-?\d+)(C|℃)～(?P<最低温>\-?\d+)(C|℃)'
df5_3 = pd.concat([df5_3, df5_3['气温'].str.extract(pat1)[["最高温","最低温"]]],1)
df5_3=df5_3.dropna()
df5_3['温差'] = df5_3['最高温'].astype(int)-df5_3['最低温'].astype(int)

# 字符串包含沙、雾、雨、雪，寻找晴
df5_3['沙'] = df5_3['天气'].str.contains('沙')
df5_3['雾'] = df5_3['天气'].str.contains('雾')
df5_3['雨'] = df5_3['天气'].str.contains('雨')
df5_3['雪'] = df5_3['天气'].str.contains('雪')
df5_3['晴'] = df5_3['天气']=='晴'
df5_3
# 正则获取雨、雪的量
# 东南西北正则

,日期,天气,气温,风向,最高温,最低温,温差,沙,雾,雨,雪,晴
0,2018-01-01,多云,1C～-4C,东南风 微风,1,-4,5,False,False,False,False,False
1,2018-01-02,阴转多云,8C～0C,东北风 3-4级,8,0,8,False,False,False,False,False
2,2018-01-03,阴转小雪,1C～-1C,东北风 4-5级转4-5级,1,-1,2,False,False,False,True,False
3,2018-01-04,阴,0C～-4C,东北风转北风 3-4级转3-4级,0,-4,4,False,False,False,False,False
4,2018-01-05,阴转多云,3C～-4C,西风转北风 3-4级转3-4级,3,-4,7,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
724,2019-12-27,多云转晴,6℃～-1℃,西南风转南风 3-4级转3-4级,6,-1,7,False,False,False,False,False
725,2019-12-28,多云转小雨,10℃～4℃,西南风转南风 3-4级转3-4级,10,4,6,False,False,True,False,False
726,2019-12-29,多云,11℃～2℃,西南风转北风 <3级转<3级,11,2,9,False,False,False,False,False
727,2019-12-30,阴转晴,4℃～-6℃,东北风转北风 4-5级转4-5级,4,-6,10,False,False,False,False,False


In [ ]:
df5_a_m.merge(df5_3, left_on='Time', right_on='日期', how='left')

In [ ]:
* 对df的水压一列构造如下时序特征：

    * 当前时刻该站点水压与本月的相同整点时间该站点水压均值的差，例如当前时刻为2018-05-20 17:00:00，那么对应需要减去的值为当前月所有17:00:00时间点水压值的均值
    * 当前时刻所在周的周末该站点水压均值与工作日水压均值之差
    * 当前时刻向前7日内，该站点水压的均值、标准差、0.95分位数、下雨天数与下雪天数的总和
    * 当前时刻向前7日内，该站点同一整点时间水压的均值、标准差、0.95分位数
    * 当前时刻所在日的该站点水压最高值与最低值出现时刻的时间差

In [ ]:
# 又一个累计求和，不包括当前的函数。需要按时间排序

In [ ]:
# 日期获取是否周末

In [ ]:
# 前七日

In [ ]:
# 前七日、同一整点group

In [ ]:
# 日group